# Simple Reinforcement Learning in Tensorflow Part 1.5:

## The Contextual Bandits

이 튜토리얼은 contextual bandit 문제를 풀수 있는 정책 그라디언트 기반 에이전트를 만드는 방법의 단순한 예제를 담고 있다.

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

## The Contextual Bandits

여기서 우리는 contextual bandits를 정의한다. 이 예제에서, 우리는 4개의 손잡이를 가진 3개의 슬롯머신을 사용한다. 이것은 각 슬롯머신은 당길 수 있는 네 개의 손잡이가 있다는 것이다. 각 슬롯머신은 각 손잡이에 대해 다른 성공 확률을 가지고 있고, 이와 같이 가장 좋은 결과를 얻기 위해 다른 행동이 필요하다. pullBandit 함수는 0의 평균을 갖는 정규 분포로부터 무작위 수를 만든다. 슬롯머신의 수가 낮을 수록, 더 자주 양의 보상을 반환할 것이다. 우리는 우리의 에이전트가 현재의 슬롯머신에서, 가장 자주 양의 보상을 주는 슬롯머신 손잡이를 항상 선택하는 것을 배우기를 원한다.

In [3]:
class contextual_bandit():
    
    def __init__(self):
        # 초기 상태는 0이다. 첫번째 슬롯머신에 앉음을 의미한다.
        self.state = 0
        
        # 우리의 슬롯머신들. 각각 손잡이 4, 2, 1이 최적이다.
        
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        # 어떤 슬롯머신의 앉을지를 랜덤하게 고른다.
        self.state = np.random.randint(0,len(self.bandits)) # 각 에피소드에 대해 다른 상태를 반환한다. (다른 슬롯머신에 앉는다)
        return self.state
        
    def pullArm(self,action):
        # 랜덤한 숫자를 얻고 확률이 그보다 작으면 양의 보상, 높으면 음의 보상을 얻는다.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

## The Policy-Based Agent

아래의 코드는 단순한 신경망 에이전트를 설정한다. 이것은 현재의 상태를 입력으로 받고, 행동을 반환한다. 이것은 환경의 상태에 기반하여 행동들을 선택하는 것을 허락한다. 완전한 강화학습 문제들을 풀 수 있게 되도록 향하게 하는 중요한 단계이다. 이 에이전트는 가중치의 집합 하나를 이용한다. 각 값은 주어진 슬롯머신의 특정 손잡이를 선택함으로부터 얻는 보상의 값의 추정치이다. 우리는 이 선택된 행동에 대한 값이 받은 보상을 향하도록 에이전트를 업데이트 하기 위해 정책 그라디언트 방법을 사용한다.

In [4]:
class agent():
    def __init__(self, lr, s_size,a_size):
        
        # 주어진 상황을 가지고 행동을 제시하는 에이전트이다.
        # 상태를 받는 부분 (어느 슬롯머신에 앉았는지를 나타냄)
        self.state_in= tf.placeholder(shape=[1],dtype=tf.int32)
        # 슬롯머신을 one hot encoding( dummy variable)화함
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        # 가중치를 가지고 행동에 대한 값을 낸다.
        output = slim.fully_connected(state_in_OH,a_size,\
            biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
        # 가중치를 가지고 손잡이의 인덱스를 선택한다.
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.argmax(self.output,0)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        # 다음 6줄은 학습 절차를 설정한다. 보상과 선택된 행동을 신경망에 보내 손실을 계산하고 신경망을 업데이트하는데
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

## Traning the Agent

환경으로부터 상태를 얻는 것, 행동을 취하는 것, 보상을 받는 것으로써 우리의 에이전트를 학습할 것이다. 이 세가지를 이용해, 우리는 주어진 상태에서 시간에 걸쳐 가장 높은 보상을 산출할 선택을 가장 자주 선택하기 위해 우리의 신경망을 적절히 업데이트 하는 방법을 알게 될 것이다.

In [5]:
tf.reset_default_graph() # 텐서플로 그래프를 초기화한다

cBandit = contextual_bandit() # 슬롯머신 들을 불러온다.
myAgent = agent(lr=0.001,s_size=cBandit.num_bandits,a_size=cBandit.num_actions) # 에이전트를 불러온다.
weights = tf.trainable_variables()[0] # 이 가중치들은 신경망을 보기위해 평가될 것이다.

total_episodes = 10000 # 에이전트를 학습할 총 에피소드를 고른다. 
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) # 슬롯머신에 대한 스코어보드를 0으로 초기화한다
e = 0.1 # 랜덤 행동을 선택할 가능성을 설정한다

init = tf.global_variables_initializer()

config = tf.ConfigProto(
        device_count={'GPU': 0}  # uncomment this line to force CPU
    )
    
# 텐서플로를 실행한다
with tf.Session(config=config) as sess:
    # 모든 변수를 초기화해준다.
    sess.run(init)
    # 에피소드 수를 기록
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() # 환경(3대의 슬롯머신)으로부터 상태(특정 슬롯머신)을 선택한다.
        
        # 행동을 신경망으로부터 선택하거나 랜덤하게 선택한다.
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
        
        reward = cBandit.pullArm(action) # 주어진 슬롯머신에서 행동을 취해 보상을 얻는다
        
        # 신경망을 업데이트한다 
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        # 보상에 대해 기록한다 
        total_reward[s,action] += reward
        # 500번 마다 각 슬롯머신의 평균 보상을 나타낸다.
        if i % 500 == 0:
            print( "Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        # 에피소드 수를 늘린다.
        i+=1
# 각 슬롯머신에 대해 최적 손잡이를 나타내고, 맞았는지 평가한다.        
for a in range(cBandit.num_bandits):
    print ("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print ("...and it was right!")
    else:
        print ("...and it was wrong!")

Mean reward for each of the 3 bandits: [ 0.25  0.    0.  ]
Mean reward for each of the 3 bandits: [ 37.25  33.    34.5 ]
Mean reward for each of the 3 bandits: [ 73.75  75.5   68.  ]
Mean reward for each of the 3 bandits: [ 116.75  111.    100.5 ]
Mean reward for each of the 3 bandits: [ 153.25  152.5   130.5 ]
Mean reward for each of the 3 bandits: [ 192.5   186.25  168.5 ]
Mean reward for each of the 3 bandits: [ 231.25  225.25  205.75]
Mean reward for each of the 3 bandits: [ 277.75  259.75  235.75]
Mean reward for each of the 3 bandits: [ 310.75  298.    273.  ]
Mean reward for each of the 3 bandits: [ 351.    334.    310.75]
Mean reward for each of the 3 bandits: [ 391.25  370.    345.5 ]
Mean reward for each of the 3 bandits: [ 431.75  409.75  376.25]
Mean reward for each of the 3 bandits: [ 470.25  447.    412.  ]
Mean reward for each of the 3 bandits: [ 503.75  488.25  446.75]
Mean reward for each of the 3 bandits: [ 543.75  528.    479.  ]
Mean reward for each of the 3 bandits